In [7]:
import numpy as np
import scipy.stats as st
import sklearn.linear_model as lm
import matplotlib.pyplot as plt
import pandas as pd
from io import StringIO
%matplotlib inline

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupShuffleSplit

In [13]:
CRCdata = pd.read_csv('/Users/teeraponsahwangarrom/Downloads/CRC-Export.txt')

In [14]:
CRCdata

,FileID,PatientID,TNAU,Analysis_Date,150.0233,150.0413,150.9795,151.0252,151.0394,151.0499,...,1450.9829,1451.9937,1452.9992,1463.9826,1469.0513,1471.0690,1485.9135,1486.9118,1487.9258,1489.9399
0,20181114_JLA443_NORMAL_MS_CUT_2.raw,JLA443,n,2018-11-01,0.0,179.500000,0.0,106.500000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20181114_JLA443_NORMAL_MS_CUT_2.raw,JLA443,n,2018-11-01,0.0,199.714286,0.0,76.142857,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20181114_JLA443_TUMOUR_MS.raw,JLA443,t,2018-11-01,0.0,166.666667,103.5,0.000000,118.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20181114_JLA445_NORMAL.raw,JLA445,n,2018-11-01,0.0,399.750000,0.0,142.750000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20181114_JLA445_NORMAL.raw,JLA445,n,2018-11-01,0.0,253.800000,0.0,153.800000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,JLA650_P_201119_CUT3.raw,JLA650,a,2019-11-01,0.0,5787.000000,0.0,863.800000,2192.40,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
800,JLA650_P_201119_CUT4.raw,JLA650,a,2019-11-01,0.0,3819.000000,0.0,0.000000,2004.80,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
801,JLA650_P_201119_CUT4.raw,JLA650,a,2019-11-01,0.0,4134.200000,0.0,0.000000,2229.80,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
802,JLA650_P_201119_CUT4.raw,JLA650,a,2019-11-01,0.0,5420.800000,0.0,0.000000,2369.20,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
extraFile = '/Users/teeraponsahwangarrom/Downloads/ExtraAnno.txt'
extra = pd.read_csv(filepath_or_buffer=extraFile, sep=",")

In [10]:
for n in range(1,len(RawData)):
    fn = RawData['Filename'][n]
    fx = fn.rfind("@")
    
    nn = fn[fx+1:len(fn)]
    eq = extra['File Name'] == nn
    if sum(eq) == 1:
        print(nn)
        
        i = [i for i, x in enumerate(eq) if x]
        
        tmp1 = extra['File Name'][i[0]]
        tmp2 = extra['Cancer'][i[0]]
        
        if tmp2 == 'Normal':
            RawData['Normal'][n] = 1
        elif tmp2 == 'Cancer':
            RawData['Cancer'][n] = 1

IKB283_T_FZ_20160914_IKNIFE_CUT_10W.raw
IKB429_N_FZ_20160914_IKNIFE_30W.raw
IKB429_N_FZ_20160914_LASER_IPA_8W.raw
IKB494_N_FZ_20160914_IKNIFE_CUT_30W.raw
IKB494_N_FZ_20160914_LASER_IPA_VENTURI_8W.raw
IKB512_N_FZ_20160914_IKNIFE_CUT_30W.raw
IKB512_N_FZ_20160914_LASER_IPA_VENTURI_8W.raw
IKB512_T_FZ_20160914_IKNIFE_CUT_10W.raw
IKB512_T_FZ_20160914_LASER_IPA_VENTURI_8W.raw
IKB523_N_FZ_20160914_IKNIFE_CUT_30W.raw
IKB523_N_FZ_20160914_LASER_IPA_VENTURI_8W.raw
IKB534_N_FZ_20160914_IKNIFE_CUT_30W.raw
IKB534_N_FZ_20160914_LASER_IPA_VENTURI_8W.raw
IKB541_T_FZ_20160914_IKNIFE_CUT_30W.raw
IKB541_T_FZ_20160914_LASER_IPA_VENTURI_8W.raw
IKB598_N_FZ_20160914_IKNIFE_CUT_30W.raw
IKB598_N_FZ_20160914_LASER_IPA_VENTURI_8W.raw
IKB97_T_FZ_20160914_IKNIFE_CUT_10W.raw
IKB97_T_FZ_20160914_LASER_IPA_VENTURI_8W.raw
0502254A_T_FZ_20161121_DOLORES_IKNIFE_CUT.raw
0502254A_T_FZ_20161121_DOLORES_LASER.raw
0602102A_T_FZ_20161121_DOLORES_IKNIFE_CUT.raw
0602102A_T_FZ_20161121_DOLORES_LASER.raw
072053A_T_FZ_20161121_DOLO

/Users/teeraponsahwangarrom/opt/anaconda3/envs/bioinfo/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/teeraponsahwangarrom/opt/anaconda3/envs/bioinfo/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


OC_002_20160108_D_CT.raw
OC_004_20160112_A_CT.raw
OC_008_A_20160315_CT.raw
OC_009_20160414_A_CT.raw
OC_010_20160414_A_CT.raw
OC_018_20160526_A_CT.raw
OC_019_20160526_B_CT.raw
OC_019_20160526_D_CT.raw
OC_020_20160526_D_CT.raw
OC_020_20160526_D_CTACTUAL.raw
OC_021_20160526_A_CT.raw
OC_025_20160809_A_CT.raw
OC_026_20160810_A_CT.raw
OC_028_A_CT.raw
OC_033_20160915_A_CT.raw
OC_034_20160920_A_CT.raw
OC_037_20161013_A_CT.raw
OC_038_20161013_A_CT.raw
OC_038_20161013_C_CT.raw
OC_040_20161018_D_CT.raw
OC_041_20161018_A_CT.raw
OC_044_20170105_B_CT.raw
OC_045_20170105_A_CT.raw
OC_045_20170105_C_CT.raw
OC_049_20170124_A_CT.raw
OC_049_20170124_B_CT.raw
OC_058_20170207_A_CT.raw
OC_058_20170207_C_CT.raw
OC_059_20170209_A_CT.raw
OC_059_20170209_B_CT.raw
OC_062_20170214_A_CT.raw
OC_066_20170221_A_CT.raw
OC_076_20170405_A_CT.raw
OC_077_20170405_A_CT.raw
OC_078_20170411_A_CT.raw
OC_079_20170411_A_CT.raw
OC_079_20170411_B_CT.raw
OVARIAN_HH12105_20150209_MOLLY_IPA.raw
OVARIAN_T13067FT_20150121_MOLLY_IPA.raw

In [15]:
for n in range(1,len(RawData)):
    fn = RawData['Filename'][n]
    fx = fn.rfind("@")
    
    nn = fn[fx+1:len(fn)]
    eq = CRCdata['FileID'] == nn
    if sum(eq) == 1:
        print(nn)
        
        i = [i for i, x in enumerate(eq) if x]
        
        tmp1 = CRCdata['FileID'][i[0]]
        tmp2 = CRCdata['TNAU'][i[0]]
        
        if tmp2 == 'n':
            RawData['Normal'][n] = 1
        elif tmp2 == 'c':
            RawData['Cancer'][n] = 1

20181114_JLA443_TUMOUR_MS.raw
JLA056_20150309_NORMAL.raw
JLA089_20150424_NORMAL.raw
JLA092_20150430_TUMOUR.raw
JLA095_20150511_NORMAL.raw
JLA174_20150930_NORMAL_04.raw
JLA184_20151009_TUMOUR2.raw
JLA217_20151210_TUMOUR.raw
JLA218_20151211_TUMOUR.raw
JLA221_20160106_NORMAL_2ND_PIECE.raw
JLA223_20160108_NORMAL.raw
JLA224_20160108_NORMAL.raw
JLA002_20141127_NORMAL_MUCOSA_IPA_RECTUM_01.raw
JLA004_20141203_TUMOUR_DOLORES_01.raw
JLA007_20141209_RECTUM_01.raw
JLA008_20141210_RIGHTCOLON_01.raw
JLA015_20150119_COLON_NORMAL_S12B.raw
JLA019_20150120_NORMAL_RECTUM_S3B.raw
JLA028_20150205_NORMAL_MUCOSA.raw
JLA032_20150203_NORMAL_RECTUM.raw
JLA034M_20150206_NORMAL_CORE.raw
JLA034M_20150206_TUMOUR_CORE.raw
JLA043M_20150220_NORMAL_B.raw
JLA043M_20150220_POLYP_B.raw
JLA056_20150309_NORMAL.raw
JLA078_011118_N_CUT1.raw
JLA089_20150424_NORMAL.raw
JLA092_20150430_TUMOUR.raw
JLA095_20150511_NORMAL.raw
JLA163_T_20180727_LAS_1.raw
JLA170_20150923_POLYP.raw
JLA174_20150930_NORMAL_04.raw
JLA176_N_130219_CUT1.ra

/Users/teeraponsahwangarrom/opt/anaconda3/envs/bioinfo/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()



JLA602_P_130219_CUT1.raw
20181114_JLA443_TUMOUR_MS.raw
JLA002_20141127_NORMAL_MUCOSA_IPA_RECTUM_01.raw
JLA004_20141203_TUMOUR_DOLORES_01.raw
JLA007_20141209_RECTUM_01.raw
JLA008_20141210_RIGHTCOLON_01.raw
JLA015_20150119_COLON_NORMAL_S12B.raw
JLA019_20150120_NORMAL_RECTUM_S3B.raw
JLA028_20150205_NORMAL_MUCOSA.raw
JLA032_20150203_NORMAL_RECTUM.raw
JLA034M_20150206_NORMAL_CORE.raw
JLA034M_20150206_TUMOUR_CORE.raw
JLA043M_20150220_NORMAL_B.raw
JLA043M_20150220_POLYP_B.raw
JLA056_20150309_NORMAL.raw
JLA078_011118_N_CUT1.raw
JLA089_20150424_NORMAL.raw
JLA092_20150430_TUMOUR.raw
JLA095_20150511_NORMAL.raw
JLA147_N_201119_CUT_1.raw
JLA163_T_20180727_LAS_1.raw
JLA170_20150923_POLYP.raw
JLA174_20150930_NORMAL_04.raw
JLA176_N_130219_CUT1.raw
JLA176_N_260619_CUT1.raw
JLA184_20151009_TUMOUR2.raw
JLA193_N_260619_CUT1.raw
JLA195_011118_N_CUT1.raw
JLA195_T_130219_CUT1.raw
JLA204_N_270718_LAS_2.raw
JLA204_T_270718_LAS_1.raw
JLA204_T_270718_LAS_2.raw
JLA210_T_TRUE_20180813_CUT_1.raw
JLA211_N_021319_CU

In [9]:
fileName = '/Users/teeraponsahwangarrom/Downloads/OneDrive_1_03-08-2020/_GlobalExportFullPartEnlarged'
i = range(1,36)
for n in range(0,len(i)):
    
    tmpFileName = "%s%d.txt" % (fileName, i[n])
    print(tmpFileName)
    
    tmpRawData = pd.read_csv(filepath_or_buffer=tmpFileName, sep=';')
    tmpRawData.index = tmpRawData['Filename']
    
    if n == 0:
        RawData = tmpRawData
    
    else:
        RawData = pd.concat([RawData, tmpRawData])

/Users/teeraponsahwangarrom/Downloads/OneDrive_1_03-08-2020/_GlobalExportFullPartEnlarged1.txt
/Users/teeraponsahwangarrom/Downloads/OneDrive_1_03-08-2020/_GlobalExportFullPartEnlarged2.txt
/Users/teeraponsahwangarrom/Downloads/OneDrive_1_03-08-2020/_GlobalExportFullPartEnlarged3.txt
/Users/teeraponsahwangarrom/Downloads/OneDrive_1_03-08-2020/_GlobalExportFullPartEnlarged4.txt
/Users/teeraponsahwangarrom/Downloads/OneDrive_1_03-08-2020/_GlobalExportFullPartEnlarged5.txt
/Users/teeraponsahwangarrom/Downloads/OneDrive_1_03-08-2020/_GlobalExportFullPartEnlarged6.txt
/Users/teeraponsahwangarrom/Downloads/OneDrive_1_03-08-2020/_GlobalExportFullPartEnlarged7.txt
/Users/teeraponsahwangarrom/Downloads/OneDrive_1_03-08-2020/_GlobalExportFullPartEnlarged8.txt
/Users/teeraponsahwangarrom/Downloads/OneDrive_1_03-08-2020/_GlobalExportFullPartEnlarged9.txt
/Users/teeraponsahwangarrom/Downloads/OneDrive_1_03-08-2020/_GlobalExportFullPartEnlarged10.txt
/Users/teeraponsahwangarrom/Downloads/OneDrive_1_

In [16]:
type(RawData)

pandas.core.frame.DataFrame

In [17]:
RawData.to_csv('/Users/teeraponsahwangarrom/Desktop/NNewAnnot_FullRawData_Project2.csv', index=False, header=True)

In [5]:
IKB_data = RawData.iloc[:,40:]

tmp = IKB_data.columns.values
mz = tmp.astype(np.float)

In [6]:
IKB_data

,150.0420,150.9820,151.0330,151.0400,151.0490,151.0870,151.1380,151.9830,152.0380,152.0440,...,979.7350,980.7680,981.7480,986.7620,988.5320,988.7730,990.7870,994.6280,996.6510,997.2500
Filename,,,,,,,,,,,,,,,,,,,,,
Z:@iKnife - Data only@Brain@Calf brain data@20161123_CALFBRAIN_NEG_IPA_HGASPIKE_2.raw,1437.3333,100.3333,34943.3333,15178.6667,3963.0000,1665.6667,37.6667,116.3333,2456.3333,2300.6667,...,638.6667,579.0000,1403.3333,683.6667,540.0000,930.0000,567.3333,1193.3333,967.0000,429.0000
Z:@iKnife - Data only@Brain@Calf brain data@20161123_CALFBRAIN_NEG_IPA_WHITE1.raw,2668.3750,595.6250,155468.5000,110238.1250,15182.7500,12259.1250,554.6250,295.8750,9904.8750,6790.6250,...,12763.1250,8750.3750,49920.0000,11210.7500,1610.1250,10415.1250,14250.8750,3923.2500,5980.5000,1374.5000
Z:@iKnife - Data only@Brain@Calf brain data@20161123_CALFBRAIN_NEG_NOIPA_GREY1.raw,1485.8696,12.4783,1341.6957,4719.8696,4736.0000,208.6087,23.6957,11.2609,153.6957,291.9565,...,60.3913,58.0435,61.0870,45.5217,72.7391,43.2174,52.5217,53.0435,57.7391,43.9130
Z:@iKnife - Data only@Brain@Calf brain data@20161123_CALFBRAIN_NEG_NOIPA_GREY2.raw,60.7692,2.3846,100.9615,1529.8846,1571.6154,93.6923,11.5000,6.2692,51.8077,91.1923,...,22.2308,21.3077,27.7692,23.1154,31.0385,19.6154,22.3077,33.3462,26.5769,19.1538
Z:@iKnife - Data only@Brain@Calf brain data@20161123_CALFBRAIN_NEG_NOIPA_GREY3.raw,49.3030,0.0303,46.5758,612.0606,641.0000,34.7576,3.6970,0.1212,16.3333,38.4545,...,22.4242,23.0303,21.7879,23.0000,24.8485,19.9697,21.0909,23.4848,21.4545,14.1818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z:@iKnife - Data only@Gynae@Uterine@Frozen@Raw Files@DM_126_20161221_CT.raw,313.8750,90.5000,147.2500,210.5000,373.7500,492.1250,72.3750,77.5000,265.5000,387.3750,...,419.2500,305.8750,431.0000,492.2500,926.3750,449.8750,312.1250,680.8750,676.1250,195.7500
Z:@iKnife - Data only@Gynae@Uterine@Frozen@Raw Files@DM_127_20161221_CT.raw,432.0000,117.3333,172.5000,263.6667,400.3333,517.5000,103.8333,105.0000,324.0000,483.1667,...,416.3333,369.3333,407.8333,477.1667,898.3333,416.5000,292.3333,811.8333,752.8333,209.8333
Z:@iKnife - Data only@Gynae@Uterine@Frozen@Raw Files@DM_128_20161221_CT.raw,428.8571,111.1429,161.0000,256.1429,423.1429,579.4286,74.8571,71.0000,386.7143,566.0000,...,305.1429,247.7143,312.4286,351.4286,916.4286,306.5714,261.0000,574.0000,547.8571,156.4286
